In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain import OpenAI, VectorDBQA
from langchain.chains import RetrievalQAWithSourcesChain
import pandas as pd
import os
from dotenv import load_dotenv # openai api key 입력을 위해서 필요

In [7]:
OPENAI_API_KEY_PATH = "/Users/eugene/Dropbox/0_Dev/07_LLM/project/env_folder/tonchat_key/.env"
# 셋째, .env 파일을 현재 실행환경에 등록. 그 결과, 개별 API 함수의 인자에 KEY값을 일일이 넣지 않아도 됨
load_dotenv(OPENAI_API_KEY_PATH)
# 넷째, 현재 실행환경에서 "OPENAI_API_KEY"라고 명명된 값을 불러오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [21]:
loader = DirectoryLoader("./faiss/", loader_cls=PyPDFLoader)
pages = loader.load_and_split()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

embeddings = OpenAIEmbeddings()


In [22]:
db = FAISS.from_documents(docs, embeddings)

# similarity search

In [23]:
query = 'fee'
docs_result = db.similarity_search(query)
docs_result

[Document(page_content="24. 1. 4. 오전  10:49 L2 fee - (current) User Guide\nhttps://tokamaknetwork.gitbook.io/home/02-service-guide/titan/user-guide/l2-fee 1/1L2 fee\nThis document outlines the calculation process for transaction fees on Titan, emphasizing their lower\ncost compared to Ethereum.\nTitan's transaction fees are determined by adding two components together:\nL2 execution fee\nL1 security fee\nL2 execution fee\ntx.gasPrice * l2GasUsed\nThe L2 execution fee refers to the fee incurred when a user's transaction is processed within the\nTitan layer2 network. It follows a calculation method shown above, identical to the standard\nEthereum transaction fee computation. However, the distinction lies in the fact that the gas price on\nTitan is significantly lower compared to Ethereum.\nL1 security fee\nl1_base_fee * (tx_data_gas + overhaed) * scalar\nThe L1 security fee is an essential fee required in the context of Optimistic Rollup. With Optimistic\nRollup, all L2 transactions are 

In [35]:
retriever = db.as_retriever()
# OpenAI(model="gpt-3.5-turbo-instruct") 처럼 ()안에 사용할 모델을 명시하지 않으면 text-davinci-003을 사용하는데 legacy model이라 지원안됨
# 따라서 OpenAI(model="gpt-3.5-turbo-instruct") 처럼 ()안에 사용할 모델을 명시해야 함
# 지시사항을 수행하는 용도인 InstructGPT을 선택해야 하며, 대화용도의 ChatGPT 모델을 선택하면 안됨
model = RetrievalQAWithSourcesChain.from_chain_type(llm=OpenAI(model="gpt-3.5-turbo-instruct"), chain_type="stuff", retriever=retriever)

In [36]:
question = "metamask에 titan 네트워크를 추가하는 방법은? "

# model() 메소드는 langchain 라이브러리의 RetrievalQAWithSourcesChain 클래스의 인스턴스 메서드
# 질의응답, context 등은 dictionary 자료형임
response = model({"question": question}, return_only_outputs=True)
print("Answer: ", response['answer'])
print("Source", response['sources'])

Answer:   현재 메타마스크에서는 타이탄 네트워크를 추가할 수 없습니다. 다른 네트워크를 선택해 주세요.

Source faiss/Add Titan Network in Metamask_KR.pdf


# Saving & loading

In [37]:
# 폴더명 지정
save_directory = "store"
# save_local 메소드는 "store" 폴더가 없으면 erro 출력대신 해당폴더를 생성한다는 장점이 있다
db.save_local(save_directory)

In [39]:
# saving, loading 둘다 폴더명을 인자로 받는다는 점을 주의해야 함 (파일명 x)
new_db = FAISS.load_local(save_directory, embeddings)


In [62]:
type(new_db)

langchain.vectorstores.faiss.FAISS

In [63]:
print(dir(new_db))

['_FAISS__add', '_FAISS__from', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_aembed_documents', '_aembed_query', '_asimilarity_search_with_relevance_scores', '_cosine_relevance_score_fn', '_embed_documents', '_embed_query', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_max_inner_product_relevance_score_fn', '_normalize_L2', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_embeddings', 'add_texts', 'adelete', 'afrom_documents', 'afrom_embeddings', 'afrom_texts', 'amax_marginal_relevance_search', 'amax_marginal_relevance_search_by_vector',

# View documenents

In [64]:
# dataframe으로 변환하면 시각화는 식은 죽먹기 : display() 사용
def show_vstore(_store):
    vector_df = store_to_df(_store)
    display(vector_df)

# vectorstore는 dictionay 자료형
# dataframe으로 변환하기 위해서는 key값을 column으로 변환해야 함
# name, page number, content 등 3가지만 key, value를 추출하여 df 만들기
def store_to_df(_store):
    v_dict = _store.docstore._dict
    data_rows = []
    for k in v_dict.keys():
        doc_name = v_dict[k].metadata['source'].split('/')[-1]
        page_number = v_dict[k].metadata['page']+1
        content = v_dict[k].page_content
        # key값과 함께 3개의 value값을 dictionary로 만들어서 data_rows에 추가
        data_rows.append({'chunk_id': k, 'document': doc_name, 'page': page_number, 'content': content})
    vector_df = pd.DataFrame(data_rows)
    return vector_df

# '''
# metadata는 faiss의 데이터 구조인 vectorstore의 일부로서, 그 안에는 사전(dictionary) 형태의 데이터 구조를 가짐
# 사전의 key, value구조를 통해 문서의 출처(source), 페이지 번호(page)와 같은 문서에 대한 중요한 정보를 포함
# 이 정보를 통해 각 데이터 항목이 어디에서 왔는지, 어떤 특성을 가지고 있는지 등을 파악할 수 있습니다.
# '''

In [65]:
show_vstore(new_db)

,chunk_id,document,page,content
0,5ec5e5c1-06cf-4854-8a4c-cd0db94d384e,Gas Estimation.pdf,1,24. 1. 4. 오전 10:59 Gas Estimation - (current)...
1,cd566fd5-f32b-4bc3-992b-22ff6fcd8a77,Gas Estimation.pdf,2,24. 1. 4. 오전 10:59 Gas Estimation - (current)...
2,1c74b1c8-e3be-4181-b948-90cb1557a529,Gas Estimation.pdf,3,24. 1. 4. 오전 10:59 Gas Estimation - (current)...
3,f81eed9b-9d78-4aa8-9b83-ad72f819bd29,Gas Estimation.pdf,4,24. 1. 4. 오전 10:59 Gas Estimation - (current)...
4,ba5d4c79-42c4-405b-b268-0c4ca76a8f83,L2 fee.pdf,1,24. 1. 4. 오전 10:49 L2 fee - (current) User Gu...
...,...,...,...,...
79,85506065-112e-405d-b082-5e5fb9a9bb13,How to Create a Standard ERC20 Token in L2.pdf,3,24. 1. 4. 오전 10:50 How to Create a Standard E...
80,40f77bcb-dff5-4801-962c-ae4a4181260e,How to Create a Standard ERC20 Token in L2.pdf,4,24. 1. 4. 오전 10:50 How to Create a Standard E...
81,0d815c3c-5080-4a0e-8310-d43dbf7c5707,How to Create a Standard ERC20 Token in L2.pdf,5,24. 1. 4. 오전 10:50 How to Create a Standard E...
82,d9699854-fe1b-4be5-9fe0-950ee302cf30,How to Create a Standard ERC20 Token in L2.pdf,6,24. 1. 4. 오전 10:50 How to Create a Standard E...
